# Evaluation Methods Demonstration

This notebook demonstrates the various evaluation methods available in the IDP library for comparing expected values with actual extraction results. It covers:

1. All evaluation methods with both match and no-match scenarios
2. Threshold testing for applicable methods
3. Edge cases:
   - Attribute not found in actual results
   - Attribute not found in expected results
   - Attribute not found in either actual or expected results

In [2]:
# First uninstall existing package (to ensure we get the latest version)
%pip uninstall -y idp_common

# Install the IDP common package with all components in development mode
%pip install -q -e "../lib/idp_common_pkg[all]"

Found existing installation: idp_common 0.3.0
Uninstalling idp_common-0.3.0:
  Successfully uninstalled idp_common-0.3.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
# Import necessary libraries
import sys
import os
import json
from typing import Dict, Any, List, Tuple, Optional
import logging

# Add parent directory to path to import the library
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger()

# Import IDP libraries
from idp_common.evaluation.models import EvaluationMethod
from idp_common.evaluation.comparator import compare_values
from idp_common.evaluation.service import EvaluationService
from idp_common.models import Document, Section, Status

print("Libraries imported successfully")

Libraries imported successfully


## Part 1: Comparing Individual Values with Different Methods

We'll test each evaluation method with matching and non-matching examples.

In [4]:
def test_comparison(method: EvaluationMethod, expected: Any, actual: Any, 
                    threshold: float = 0.8, document_class: str = "TestDoc",
                    attr_name: str = "test_attr", attr_description: str = "Test attribute"):
    """Test a comparison method and print results."""
    
    print(f"\n{'-'*60}")
    print(f"Method: {method.name}")
    print(f"Expected: {expected}")
    print(f"Actual: {actual}")
    
    if method in [EvaluationMethod.FUZZY, EvaluationMethod.BERT]:
        print(f"Threshold: {threshold}")
    
    # Set up LLM config for the LLM method
    llm_config = None
    if method == EvaluationMethod.LLM:
        llm_config = {
            "model": "us.anthropic.claude-3-7-sonnet-20250219-v1:0",
            "temperature": 0.0,
            "top_k": 250,
            "system_prompt": "You are an evaluator that helps determine if the predicted and expected values match for document attribute extraction.",
            "task_prompt": """I need to evaluate attribute extraction for a document of class: {DOCUMENT_CLASS}.

For the attribute named "{ATTRIBUTE_NAME}" described as "{ATTRIBUTE_DESCRIPTION}":
- Expected value: {EXPECTED_VALUE}
- Actual value: {ACTUAL_VALUE}

Do these values match in meaning, taking into account formatting differences, word order, abbreviations, and semantic equivalence?
Provide your assessment as a JSON with three fields:
- "match": boolean (true if they match, false if not)
- "score": number between 0 and 1 representing the confidence/similarity score
- "reason": brief explanation of your decision

Respond ONLY with the JSON and nothing else."""
        }
    
    # Perform the comparison
    matched, score, reason = compare_values(
        expected=expected,
        actual=actual,
        method=method,
        threshold=threshold,
        document_class=document_class,
        attr_name=attr_name,
        attr_description=attr_description,
        llm_config=llm_config
    )
    
    print(f"Matched: {matched}")
    print(f"Score: {score}")
    if reason:
        print(f"Reason: {reason}")
        
    return matched, score, reason

### Test 1: EXACT Method
Testing exact string matching with both match and non-match cases.

In [5]:
# EXACT method - Match
test_comparison(EvaluationMethod.EXACT, "Account #12345", "Account #12345")

# EXACT method - No match
test_comparison(EvaluationMethod.EXACT, "Account #12345", "Account #12346")

# EXACT method - Match with different casing and punctuation
test_comparison(EvaluationMethod.EXACT, "Account Number: 12345", "account number 12345")


------------------------------------------------------------
Method: EXACT
Expected: Account #12345
Actual: Account #12345
Matched: True
Score: 1.0

------------------------------------------------------------
Method: EXACT
Expected: Account #12345
Actual: Account #12346
Matched: False
Score: 0.0

------------------------------------------------------------
Method: EXACT
Expected: Account Number: 12345
Actual: account number 12345
Matched: True
Score: 1.0


(True, 1.0, None)

### Test 2: NUMERIC_EXACT Method
Testing numeric comparison with different formats.

In [6]:
# NUMERIC_EXACT method - Match
test_comparison(EvaluationMethod.NUMERIC_EXACT, "$1,250.00", 1250)

# NUMERIC_EXACT method - No match
test_comparison(EvaluationMethod.NUMERIC_EXACT, "$1,250.00", 1251)

# NUMERIC_EXACT method - Match with different formats
test_comparison(EvaluationMethod.NUMERIC_EXACT, "(1,250.00)", "-1250")


------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: $1,250.00
Actual: 1250
Matched: True
Score: 1.0

------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: $1,250.00
Actual: 1251
Matched: False
Score: 0.0

------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: (1,250.00)
Actual: -1250
Matched: False
Score: 0.0


(False, 0.0, None)

### Test 3: FUZZY Method
Testing fuzzy comparison with different thresholds.

In [7]:
# FUZZY method - High match
test_comparison(EvaluationMethod.FUZZY, "John A. Smith", "John Smith", threshold=0.8)

# FUZZY method - Medium match 
matched, score, _ = test_comparison(EvaluationMethod.FUZZY, "John A. Smith", "John Simpson", threshold=0.8)
print(f"With threshold=0.6: {score >= 0.6}")

# FUZZY method - Low match
test_comparison(EvaluationMethod.FUZZY, "John Alexander Smith", "Jane Marie Johnson", threshold=0.8)


------------------------------------------------------------
Method: FUZZY
Expected: John A. Smith
Actual: John Smith
Threshold: 0.8
Matched: True
Score: 0.8333333333333334

------------------------------------------------------------
Method: FUZZY
Expected: John A. Smith
Actual: John Simpson
Threshold: 0.8
Matched: False
Score: 0.41666666666666663
With threshold=0.6: False

------------------------------------------------------------
Method: FUZZY
Expected: John Alexander Smith
Actual: Jane Marie Johnson
Threshold: 0.8
Matched: False
Score: 0.15000000000000002


(False, 0.15000000000000002, None)

### Test 4: HUNGARIAN Method
Testing list comparison using the Hungarian algorithm.

In [8]:
# HUNGARIAN method - Full match
expected_list = ["Deposit: $500", "Withdrawal: $150", "Transfer: $200"]
actual_list = ["Deposit: $500", "Transfer: $200", "Withdrawal: $150"]
test_comparison(EvaluationMethod.HUNGARIAN, expected_list, actual_list)

# HUNGARIAN method - Partial match
expected_list = ["Deposit: $500", "Withdrawal: $150", "Transfer: $200"]
actual_list = ["Deposit: $500", "Withdrawal: $150", "Transfer: $210"]
test_comparison(EvaluationMethod.HUNGARIAN, expected_list, actual_list)

# HUNGARIAN method - Different number of items
expected_list = ["Deposit: $500", "Withdrawal: $150", "Transfer: $200"]
actual_list = ["Deposit: $500", "Withdrawal: $150"]
test_comparison(EvaluationMethod.HUNGARIAN, expected_list, actual_list)

# HUNGARIAN method - Non-list values (should convert to list)
test_comparison(EvaluationMethod.HUNGARIAN, "Single item", "Single item")


------------------------------------------------------------
Method: HUNGARIAN
Expected: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $200']
Actual: ['Deposit: $500', 'Transfer: $200', 'Withdrawal: $150']
Matched: True
Score: 1.0

------------------------------------------------------------
Method: HUNGARIAN
Expected: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $200']
Actual: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $210']
Matched: False
Score: 0.6666666666666666

------------------------------------------------------------
Method: HUNGARIAN
Expected: ['Deposit: $500', 'Withdrawal: $150', 'Transfer: $200']
Actual: ['Deposit: $500', 'Withdrawal: $150']
Matched: True
Score: 1.0

------------------------------------------------------------
Method: HUNGARIAN
Expected: Single item
Actual: Single item
Matched: True
Score: 1.0


(True, 1.0, None)

### Test 5: LLM Method
Testing semantic comparison using a Large Language Model.

In [16]:
# LLM method - High semantic match (different wording, same meaning)
test_comparison(
    EvaluationMethod.LLM,
    "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
    "Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.",
    document_class="BankStatement",
    attr_name="statement_summary",
    attr_description="Summary of the bank statement"
)


------------------------------------------------------------
Method: LLM
Expected: Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.
Actual: Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.


INFO:idp_common.bedrock:Bedrock request attempt 1/8:
INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '0285655c-2491-42d6-88ad-b6345919a94d', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:38:47 GMT', 'content-type': 'application/json', 'content-length': '604', 'connection': 'keep-alive', 'x-amzn-requestid': '0285655c-2491-42d6-88ad-b6345919a94d'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{\n  "match": true,\n  "score": 0.95,\n  "reason": "Both values convey the same information about deposits ($1,250), withdrawals ($850), and ending balance ($2,400). The wording is slightly different (\'Monthly statement showing\' vs \'Statement with\') but the core financial information is identical."\n}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 218, 'outputTokens': 81, 'totalTokens': 299, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 3151}}
INFO:idp_common.evaluati

Matched: True
Score: 0.95
Reason: Both values convey the same information about deposits ($1,250), withdrawals ($850), and ending balance ($2,400). The wording is slightly different ('Monthly statement showing' vs 'Statement with') but the core financial information is identical.


(True,
 0.95,
 "Both values convey the same information about deposits ($1,250), withdrawals ($850), and ending balance ($2,400). The wording is slightly different ('Monthly statement showing' vs 'Statement with') but the core financial information is identical.")

In [17]:
# LLM method - No semantic match (different meaning)
test_comparison(
    EvaluationMethod.LLM,
    "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
    "Statement with deposits of $2,500 and withdrawals of $1,200, leaving a balance of $3,800.",
    document_class="BankStatement",
    attr_name="statement_summary",
    attr_description="Summary of the bank statement"
)

INFO:idp_common.bedrock:Bedrock request attempt 1/8:



------------------------------------------------------------
Method: LLM
Expected: Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.
Actual: Statement with deposits of $2,500 and withdrawals of $1,200, leaving a balance of $3,800.


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': 'dbb1946a-872a-4dc5-ab5a-a10e16fd03fe', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:38:55 GMT', 'content-type': 'application/json', 'content-length': '675', 'connection': 'keep-alive', 'x-amzn-requestid': 'dbb1946a-872a-4dc5-ab5a-a10e16fd03fe'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{\n  "match": false,\n  "score": 0.2,\n  "reason": "The values contain completely different financial figures. Expected shows deposits of $1,250, withdrawals of $850, and ending balance of $2,400, while actual shows deposits of $2,500, withdrawals of $1,200, and balance of $3,800. These are fundamentally different statements describing different financial situations."\n}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 219, 'outputTokens': 98, 'totalTokens': 317, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 2950}}
INFO:idp_c

Matched: False
Score: 0.2
Reason: The values contain completely different financial figures. Expected shows deposits of $1,250, withdrawals of $850, and ending balance of $2,400, while actual shows deposits of $2,500, withdrawals of $1,200, and balance of $3,800. These are fundamentally different statements describing different financial situations.


(False,
 0.2,
 'The values contain completely different financial figures. Expected shows deposits of $1,250, withdrawals of $850, and ending balance of $2,400, while actual shows deposits of $2,500, withdrawals of $1,200, and balance of $3,800. These are fundamentally different statements describing different financial situations.')

In [18]:
# LLM method - Partial semantic match (some differences)
test_comparison(
    EvaluationMethod.LLM,
    "Policy effective date: January 15, 2023 to January 14, 2024",
    "Policy period begins on Jan 15, 2023 and expires on Jan 15, 2024",
    document_class="InsurancePolicy",
    attr_name="policy_period",
    attr_description="The dates during which the insurance policy is effective"
)

INFO:idp_common.bedrock:Bedrock request attempt 1/8:



------------------------------------------------------------
Method: LLM
Expected: Policy effective date: January 15, 2023 to January 14, 2024
Actual: Policy period begins on Jan 15, 2023 and expires on Jan 15, 2024


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '0a4215ff-98f8-42b4-af2e-9e145bd3b8b4', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:39:03 GMT', 'content-type': 'application/json', 'content-length': '616', 'connection': 'keep-alive', 'x-amzn-requestid': '0a4215ff-98f8-42b4-af2e-9e145bd3b8b4'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{\n  "match": false,\n  "score": 0.9,\n  "reason": "The values are very similar but have a critical difference in the end date. The expected value states January 14, 2024 as the end date, while the actual value states Jan 15, 2024. This is a one-day difference that could be significant for insurance coverage."\n}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 215, 'outputTokens': 89, 'totalTokens': 304, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 1979}}
INFO:idp_common.evaluation.comparator:LLM evaluation for policy_perio

Matched: False
Score: 0.9
Reason: The values are very similar but have a critical difference in the end date. The expected value states January 14, 2024 as the end date, while the actual value states Jan 15, 2024. This is a one-day difference that could be significant for insurance coverage.


(False,
 0.9,
 'The values are very similar but have a critical difference in the end date. The expected value states January 14, 2024 as the end date, while the actual value states Jan 15, 2024. This is a one-day difference that could be significant for insurance coverage.')

## Part 2: Edge Cases - Missing Attributes

### Test 6: Attributes Not Found
Testing scenarios where attributes are missing.

In [10]:
# Case 1: Attribute not found in actual (expected exists, actual is None)
print("\nCase 1: Attribute not found in actual results")
for method in [EvaluationMethod.EXACT, EvaluationMethod.NUMERIC_EXACT, 
               EvaluationMethod.FUZZY, EvaluationMethod.LLM]:
    matched, score, reason = test_comparison(method, "This value exists", None)
    print(f"Method: {method.name} - Score: {score} - Matched: {matched}")

# Case 2: Attribute not found in expected (expected is None, actual exists)
print("\nCase 2: Attribute not found in expected results")
for method in [EvaluationMethod.EXACT, EvaluationMethod.NUMERIC_EXACT, 
               EvaluationMethod.FUZZY, EvaluationMethod.LLM]:
    matched, score, reason = test_comparison(method, None, "This value exists")
    print(f"Method: {method.name} - Score: {score} - Matched: {matched}")

# Case 3: Attribute not found in either (both are None)
print("\nCase 3: Attribute not found in either expected or actual results")
for method in [EvaluationMethod.EXACT, EvaluationMethod.NUMERIC_EXACT, 
               EvaluationMethod.FUZZY, EvaluationMethod.LLM]:
    matched, score, reason = test_comparison(method, None, None)
    print(f"Method: {method.name} - Score: {score} - Matched: {matched}")

# Case 4: Empty string values ("")
print("\nCase 4: Empty string values")
for method in [EvaluationMethod.EXACT, EvaluationMethod.FUZZY, EvaluationMethod.LLM]:
    matched, score, reason = test_comparison(method, "", "")
    print(f"Method: {method.name} - Score: {score} - Matched: {matched}")

INFO:idp_common.bedrock:Bedrock request attempt 1/8:



Case 1: Attribute not found in actual results

------------------------------------------------------------
Method: EXACT
Expected: This value exists
Actual: None
Matched: False
Score: 0.0
Method: EXACT - Score: 0.0 - Matched: False

------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: This value exists
Actual: None
Matched: False
Score: 0.0
Method: NUMERIC_EXACT - Score: 0.0 - Matched: False

------------------------------------------------------------
Method: FUZZY
Expected: This value exists
Actual: None
Threshold: 0.8
Matched: False
Score: 0.0
Method: FUZZY - Score: 0.0 - Matched: False

------------------------------------------------------------
Method: LLM
Expected: This value exists
Actual: None


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '398b7142-39cb-4bc8-a175-47e297dd1924', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:35:12 GMT', 'content-type': 'application/json', 'content-length': '519', 'connection': 'keep-alive', 'x-amzn-requestid': '398b7142-39cb-4bc8-a175-47e297dd1924'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{\n  "match": false,\n  "score": 0.0,\n  "reason": "The expected value is \'This value exists\' while the actual value is None/missing. These clearly do not match as one contains content and the other is empty/null."\n}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 168, 'outputTokens': 65, 'totalTokens': 233, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 2755}}
INFO:idp_common.evaluation.comparator:LLM evaluation for test_attr: match=False, score=0.0, reason=The expected value is 'This value exists' while the actual value 

Matched: False
Score: 0.0
Reason: The expected value is 'This value exists' while the actual value is None/missing. These clearly do not match as one contains content and the other is empty/null.
Method: LLM - Score: 0.0 - Matched: False

Case 2: Attribute not found in expected results

------------------------------------------------------------
Method: EXACT
Expected: None
Actual: This value exists
Matched: False
Score: 0.0
Method: EXACT - Score: 0.0 - Matched: False

------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: None
Actual: This value exists
Matched: False
Score: 0.0
Method: NUMERIC_EXACT - Score: 0.0 - Matched: False

------------------------------------------------------------
Method: FUZZY
Expected: None
Actual: This value exists
Threshold: 0.8
Matched: False
Score: 0.0
Method: FUZZY - Score: 0.0 - Matched: False

------------------------------------------------------------
Method: LLM
Expected: None
Actual: This value exists


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '5554c783-37c8-4183-b02a-179696d061f7', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:35:16 GMT', 'content-type': 'application/json', 'content-length': '570', 'connection': 'keep-alive', 'x-amzn-requestid': '5554c783-37c8-4183-b02a-179696d061f7'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{\n  "match": false,\n  "score": 0.0,\n  "reason": "The expected value is None (null/empty), while the actual value contains text \'This value exists\'. These are completely different - one indicates no value was found while the other contains specific text content."\n}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 168, 'outputTokens': 70, 'totalTokens': 238, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 3147}}
INFO:idp_common.evaluation.comparator:LLM evaluation for test_attr: match=False, score=0.0, reason=The expected v

Matched: False
Score: 0.0
Reason: The expected value is None (null/empty), while the actual value contains text 'This value exists'. These are completely different - one indicates no value was found while the other contains specific text content.
Method: LLM - Score: 0.0 - Matched: False

Case 3: Attribute not found in either expected or actual results

------------------------------------------------------------
Method: EXACT
Expected: None
Actual: None
Matched: True
Score: 1.0
Method: EXACT - Score: 1.0 - Matched: True

------------------------------------------------------------
Method: NUMERIC_EXACT
Expected: None
Actual: None
Matched: True
Score: 1.0
Method: NUMERIC_EXACT - Score: 1.0 - Matched: True

------------------------------------------------------------
Method: FUZZY
Expected: None
Actual: None
Threshold: 0.8
Matched: True
Score: 1.0
Method: FUZZY - Score: 1.0 - Matched: True

------------------------------------------------------------
Method: LLM
Expected: None
Actual: N

INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': 'cf6f108e-2743-4def-8769-b28b6380995e', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:35:18 GMT', 'content-type': 'application/json', 'content-length': '430', 'connection': 'keep-alive', 'x-amzn-requestid': 'cf6f108e-2743-4def-8769-b28b6380995e'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{\n  "match": true,\n  "score": 1.0,\n  "reason": "Both the expected and actual values are None, indicating a perfect match."\n}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 166, 'outputTokens': 43, 'totalTokens': 209, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 1604}}
INFO:idp_common.evaluation.comparator:LLM evaluation for test_attr: match=True, score=1.0, reason=Both the expected and actual values are None, indicating a perfect match.
INFO:idp_common.bedrock:Bedrock request attempt 1/8:


Matched: True
Score: 1.0
Reason: Both the expected and actual values are None, indicating a perfect match.
Method: LLM - Score: 1.0 - Matched: True

Case 4: Empty string values

------------------------------------------------------------
Method: EXACT
Expected: 
Actual: 
Matched: True
Score: 1.0
Method: EXACT - Score: 1.0 - Matched: True

------------------------------------------------------------
Method: FUZZY
Expected: 
Actual: 
Threshold: 0.8
Matched: True
Score: 1.0
Method: FUZZY - Score: 1.0 - Matched: True

------------------------------------------------------------
Method: LLM
Expected: 
Actual: 


INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': 'dcac1f85-de58-4201-ba8f-02f8fadb4bca', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:35:21 GMT', 'content-type': 'application/json', 'content-length': '465', 'connection': 'keep-alive', 'x-amzn-requestid': 'dcac1f85-de58-4201-ba8f-02f8fadb4bca'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{\n  "match": true,\n  "score": 1.0,\n  "reason": "Both the expected and actual values are empty, which means they are identical and therefore match perfectly."\n}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 166, 'outputTokens': 48, 'totalTokens': 214, 'cacheReadInputTokens': 0, 'cacheWriteInputTokens': 0}, 'metrics': {'latencyMs': 2310}}
INFO:idp_common.evaluation.comparator:LLM evaluation for test_attr: match=True, score=1.0, reason=Both the expected and actual values are empty, which means they are identical and therefore match perfectly.


Matched: True
Score: 1.0
Reason: Both the expected and actual values are empty, which means they are identical and therefore match perfectly.
Method: LLM - Score: 1.0 - Matched: True


## Part 3: Full Document Evaluation

Now we'll test the full document evaluation service with different evaluation methods. We'll use the real AWS Bedrock service for LLM-based evaluations.

In [11]:
# Setup test config
test_config = {
    "classes": [
        {
            "name": "TestDocument",
            "attributes": [
                {
                    "name": "exact_match_attr",
                    "description": "Attribute for exact matching",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "numeric_attr",
                    "description": "Attribute for numeric matching",
                    "evaluation_method": "NUMERIC_EXACT"
                },
                {
                    "name": "fuzzy_attr",
                    "description": "Attribute for fuzzy matching",
                    "evaluation_method": "FUZZY",
                    "evaluation_threshold": 0.8
                },
                {
                    "name": "list_attr",
                    "description": "Attribute for list comparison",
                    "evaluation_method": "HUNGARIAN"
                },
                {
                    "name": "llm_attr",
                    "description": "Attribute for semantic comparison",
                    "evaluation_method": "LLM"
                },
                {
                    "name": "missing_in_actual",
                    "description": "Attribute missing in actual results",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "missing_in_expected",
                    "description": "Attribute missing in expected results",
                    "evaluation_method": "EXACT"
                },
                {
                    "name": "missing_everywhere",
                    "description": "Attribute missing in both expected and actual",
                    "evaluation_method": "EXACT"
                }
            ]
        }
    ],
    "evaluation": {
        "llm_method": {
            "model": "anthropic.claude-3-sonnet-20240229-v1:0",
            "temperature": 0.0,
            "top_k": 250,
            "system_prompt": "You are an evaluator for document extraction attributes.",
            "task_prompt": """I need to evaluate attribute extraction for a document of class: {DOCUMENT_CLASS}.

For the attribute named "{ATTRIBUTE_NAME}" described as "{ATTRIBUTE_DESCRIPTION}":
- Expected value: {EXPECTED_VALUE}
- Actual value: {ACTUAL_VALUE}

Do these values match in meaning, taking into account formatting differences, word order, abbreviations, and semantic equivalence?
Provide your assessment as a JSON with three fields:
- "match": boolean (true if they match, false if not)
- "score": number between 0 and 1 representing the confidence/similarity score
- "reason": brief explanation of your decision

Respond ONLY with the JSON and nothing else."""
        }
    }
}

In [12]:
# Create mock S3 retrieval function
def mock_s3_get_json(uri: str) -> Dict[str, Any]:
    """Mock S3 file retrieval."""
    if "expected" in uri:
        return {
            "exact_match_attr": "Exact Match Value",
            "numeric_attr": "$1,250.00",
            "fuzzy_attr": "John Alexander Smith",
            "list_attr": ["Item 1", "Item 2", "Item 3"],
            "llm_attr": "Monthly statement showing deposits of $1,250, withdrawals of $850, ending balance of $2,400.",
            "missing_in_actual": "This value exists in expected only",
            # missing_in_expected is intentionally omitted
            # missing_everywhere is intentionally omitted
        }
    else:  # actual results
        return {
            "exact_match_attr": "Exact Match Value",  # Exact match
            "numeric_attr": 1250,  # Numeric match
            "fuzzy_attr": "John A Smith",  # Fuzzy match
            "list_attr": ["Item 1", "Item 3", "Item 2"],  # List with different order
            "llm_attr": "Statement with deposits totaling $1,250 and withdrawals of $850, leaving a balance of $2,400.",  # Semantic match
            # missing_in_actual is intentionally omitted
            "missing_in_expected": "This value exists in actual only",
            # missing_everywhere is intentionally omitted
        }

# Set up mock storage - we'll still use this for S3
class MockS3:
    # Store report content for later display
    report_content = ""
    results_content = {}
    
    @staticmethod
    def get_json_content(uri: str) -> Dict[str, Any]:
        return mock_s3_get_json(uri)
    
    @staticmethod
    def write_content(content: Any, bucket: str, key: str, content_type: str = None):
        print(f"Writing content to s3://{bucket}/{key}")
        if key.endswith("results.json"):
            # Store the results for later access
            MockS3.results_content = content
            print(f"Evaluation results summary: {json.dumps(content.get('overall_metrics', {}), indent=2)}")
        elif key.endswith("report.md"):
            # Store the markdown report for later display
            MockS3.report_content = content

In [13]:
# Create mock documents for evaluation
def create_test_document(doc_id: str, is_expected: bool = False) -> Document:
    """Create a test document with a section."""
    section = Section(
        section_id="sec-001",
        classification="TestDocument",
        extraction_result_uri=f"s3://test-bucket/{doc_id}/{'expected' if is_expected else 'actual'}/extraction.json"
    )
    
    doc = Document(
        id=doc_id,
        sections=[section],
        input_key=doc_id,
        input_bucket="test-bucket",
        output_bucket="test-bucket",
        status=Status.EXTRACTED
    )
    
    return doc

# Create test documents
actual_doc = create_test_document("test-doc-001")
expected_doc = create_test_document("test-doc-001-baseline", is_expected=True)

In [14]:
# Evaluate document
# Only patch S3 module - use real Bedrock
import idp_common.evaluation.service
idp_common.evaluation.service.s3 = MockS3

# Create evaluation service
evaluation_service = EvaluationService(region="us-east-1", config=test_config)

# Evaluate document
result_doc = evaluation_service.evaluate_document(actual_doc, expected_doc, store_results=True)

# Print results
if hasattr(result_doc, 'evaluation_result'):
    eval_result = result_doc.evaluation_result
    print(f"\nOverall metrics: {eval_result.overall_metrics}")
    
    # Check section results
    for section_result in eval_result.section_results:
        print(f"\nSection {section_result.section_id} - Class: {section_result.document_class}")
        print(f"Metrics: {section_result.metrics}")
        
        # Print attribute details
        print("\nAttribute Details:")
        print("-" * 100)
        print(f"{'Name':<20} {'Method':<15} {'Expected':<25} {'Actual':<25} {'Matched':<10} {'Score':<10} {'Reason'}")
        print("-" * 100)
        
        for attr in section_result.attributes:
            expected_val = str(attr.expected)[:25]
            actual_val = str(attr.actual)[:25]
            method = attr.evaluation_method
            reason = attr.reason[:50] + "..." if attr.reason and len(attr.reason) > 50 else (attr.reason or "")
            print(f"{attr.name:<20} {method:<15} {expected_val:<25} {actual_val:<25} {attr.matched!s:<10} {attr.score:<10.2f} {reason}")

INFO:idp_common.evaluation.service:Initialized evaluation service with LLM configuration
INFO:idp_common.bedrock:Bedrock request attempt 1/8:
INFO:idp_common.bedrock:Response: {'ResponseMetadata': {'RequestId': '4042f684-75e7-4146-89d6-d8c79d842943', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Thu, 17 Apr 2025 18:35:23 GMT', 'content-type': 'application/json', 'content-length': '462', 'connection': 'keep-alive', 'x-amzn-requestid': '4042f684-75e7-4146-89d6-d8c79d842943'}, 'RetryAttempts': 0}, 'output': {'message': {'role': 'assistant', 'content': [{'text': '{"match": true, "score": 0.95, "reason": "The expected and actual values convey the same essential information about a monthly statement with deposits of $1,250, withdrawals of $850, and an ending balance of $2,400, despite minor differences in wording and formatting."}'}]}}, 'stopReason': 'end_turn', 'usage': {'inputTokens': 208, 'outputTokens': 69, 'totalTokens': 277}, 'metrics': {'latencyMs': 1674}}
INFO:idp_common.evaluation

Writing content to s3://test-bucket/test-doc-001/evaluation/results.json
Evaluation results summary: {
  "precision": 0.6666666666666666,
  "recall": 0.8,
  "f1_score": 0.7272727272727272,
  "accuracy": 0.625,
  "false_alarm_rate": 0.5,
  "false_discovery_rate": 0.2
}
Writing content to s3://test-bucket/test-doc-001/evaluation/report.md

Overall metrics: {'precision': 0.6666666666666666, 'recall': 0.8, 'f1_score': 0.7272727272727272, 'accuracy': 0.625, 'false_alarm_rate': 0.5, 'false_discovery_rate': 0.2}

Section sec-001 - Class: TestDocument
Metrics: {'precision': 0.6666666666666666, 'recall': 0.8, 'f1_score': 0.7272727272727272, 'accuracy': 0.625, 'false_alarm_rate': 0.5, 'false_discovery_rate': 0.2}

Attribute Details:
----------------------------------------------------------------------------------------------------
Name                 Method          Expected                  Actual                    Matched    Score      Reason
------------------------------------------------

### Display the Evaluation Report

Let's display the markdown evaluation report that was generated:

In [15]:
from IPython.display import Markdown

# Display the markdown report
if MockS3.report_content:
    display(Markdown(MockS3.report_content))
else:
    print("No evaluation report was generated.")

# Document Evaluation: test-doc-001

## Summary
- **Match Rate**: 🟠 4/8 attributes matched [██████████░░░░░░░░░░] 50%
- **Precision**: 0.67 | **Recall**: 0.80 | **F1 Score**: 🟡 0.73

## Overall Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.6667 | 🟠 Fair |
| recall | 0.8000 | 🟡 Good |
| f1_score | 0.7273 | 🟡 Good |
| accuracy | 0.6250 | 🟠 Fair |
| false_alarm_rate | 0.5000 | 🟠 Fair |
| false_discovery_rate | 0.2000 | 🟡 Good |


## Section: sec-001 (TestDocument)
### Metrics
| Metric | Value | Rating |
| ------ | :----: | :----: |
| precision | 0.6667 | 🟠 Fair |
| recall | 0.8000 | 🟡 Good |
| f1_score | 0.7273 | 🟡 Good |
| accuracy | 0.6250 | 🟠 Fair |
| false_alarm_rate | 0.5000 | 🟠 Fair |
| false_discovery_rate | 0.2000 | 🟡 Good |


### Attributes
| Status | Attribute | Expected | Actual | Score | Method | Reason |
| :----: | --------- | -------- | ------ | ----- | ------ | ------ |
| ✅ | exact_match_attr | Exact Match Value | Exact Match Value | 1.00 | EXACT |  |
| ✅ | numeric_attr | $1,250.00 | 1250 | 1.00 | NUMERIC_EXACT |  |
| ❌ | fuzzy_attr | John Alexander Smith | John A Smith | 0.60 | FUZZY (evaluation_threshold: 0.8) |  |
| ✅ | list_attr | ['Item 1', 'Item 2', 'Item 3'] | ['Item 1', 'Item 3', 'Item 2'] | 1.00 | HUNGARIAN |  |
| ✅ | llm_attr | Monthly statement showing deposits of $1,250, with | Statement with deposits totaling $1,250 and withdr | 0.95 | LLM | The expected and actual values convey the same essential information about a mon |
| ❌ | missing_in_actual | This value exists in expected only | None | 0.00 | EXACT |  |
| ❌ | missing_in_expected | None | This value exists in actual only | 0.00 | EXACT |  |
| ❌ | missing_everywhere | None | None | 1.00 | EXACT |  |


Execution time: 2.44 seconds

## Evaluation Methods Used

This evaluation used the following methods to compare expected and actual values:

1. **EXACT** - Exact string match after stripping punctuation and whitespace
2. **NUMERIC_EXACT** - Exact numeric match after normalizing
3. **FUZZY** - Fuzzy string matching using string similarity metrics (with optional evaluation_threshold)
4. **BERT** - Semantic similarity comparison using BERT embeddings (with evaluation_threshold)
5. **HUNGARIAN** - Bipartite matching algorithm for lists of values
6. **LLM** - Advanced semantic evaluation using Bedrock large language models

Each attribute is configured with a specific evaluation method based on the data type and comparison needs.

## Summary

This notebook has demonstrated:

1. All evaluation methods available in the IDP library:
   - EXACT - Exact string matching
   - NUMERIC_EXACT - Numeric value matching
   - FUZZY - Fuzzy string matching with adjustable thresholds
   - HUNGARIAN - List comparison using the Hungarian algorithm
   - LLM - Semantic comparison using Large Language Models

2. Handling of edge cases:
   - Attributes missing in actual results
   - Attributes missing in expected results
   - Attributes missing in both actual and expected results
   - Empty string values

3. Full document evaluation with mixed evaluation methods
   - Comprehensive metrics calculation
   - Detailed attribute-level results

4. Threshold sensitivity analysis for fuzzy matching
   - How different threshold values affect match results
   - Trade-offs between precision and recall